


<h1>
<hr style=" border:none; height:3px;">
<center>Exam MAP 670 Causality</center>
<hr style=" border:none; height:3px;">
</h1>

<h6><center>Exercise 1</center></h6>

# Setup

First we import the libraries.

In [1]:
import logging
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import statsmodels
import statsmodels.formula.api as smf
import statsmodels.api as sm
import graphviz as gr
from statsmodels.compat import lzip
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
%matplotlib inline



To estimate the average treatment effect under unconfoundness, we have studied the
IPW, the regression adjustment and the double robust estimators. Another estimator
based on stratification can be defined.
The principle is to form L strata based on the values of the propensity score,
which in practice consists in grouping observations for which $\widehat{e}(X)$ are similar. Then,
in each stratum l, the average treatment effect is estimated as $\overline{Y(1)}_l − \overline{Y(0)}_l$ where
$\overline{Y(w)}_l$ denotes the average value of the outcome for units with treatment w in stratum
l. Finally, the ATE estimator is the mean of the difference between means in each
stratum:
$$
\widehat{\tau}_{strat}=\frac{1}{L}\sum \overline{Y(1)}_l − \overline{Y(0)}_l
$$

The aim of this exercise is to conduct a simulation study to evaluate the pros and cons of inverse propensity
weighting and stratification.

[link text](https://)<div class="alert alert-block alert-success">

**Question 1.** Generate data as follows, for n = 400 and p = 10:

</div>


In [23]:
X = np.random.uniform(low=-1.0, high=1.0, size=(400,10))

In [24]:
propensity=0.1+0.85*np.sqrt(np.maximum(0,1+X[:,0]+ X[:,1])/3)

In [25]:
W =np.random.binomial(1, propensity, 400)

In [26]:
Y = np.multiply(W, np.maximum(0, X[:,0])) + np.exp(X[:,1] + X[:,2])

<div class="alert alert-block alert-success">

**Question 2.** Show that the average treatment effect $\tau$ of the treatment $W$ on $Y$ in this simulation design is equal to
0.25. (You can show it by simulation if you want). 

</div>

According to the simulation desigh, the average treatment effect $\tau$ equals to `maximum(0, X_0)` where `X_0` is the first component of vector X and `X_0` follows an uniform distribution [-1,1].  

The expectation of variable `maximum(0, X_0)` can be computed. Thus,

 $\tau= \frac{1}{2}*0+\frac{1}{2}* \int_0^1 x dx = 0.25$.

<div class="alert alert-block alert-success">

**Question 3.** Fit the propensity score $\widehat{e}$ via logistic regression, and estimate $\widehat{\tau}$ via $\widehat{\tau}_{IPW}$ 
</div>

In [33]:
import sklearn.linear_model as sklearn_linear_model

def clip_probabilities(prob, th=0.1) :
  prob[prob < th] = th
  prob[prob > (1-th)] = 1-th
  return prob 

propensity_model = sklearn_linear_model.LogisticRegression(random_state=0, fit_intercept=True).fit(X[:, :2], W)
estimates_prob = propensity_model.predict_proba(X[:, :2])[:, 1]
e_hat = clip_probabilities(estimates_prob)
tau_IPW = (Y * ((W/e_hat) - (1-W)/(1-e_hat))).mean()
print("Tau_hat via IPW is: ", tau_IPW)

Tau_hat via IPW is:  0.30157894411644626


<div class="alert alert-block alert-success">

**Question 4.**We define the number of strata as $L = n^\rho$ . For $\rho = 0.1, 0.2, 0.3, 0.4$, compute
$\widehat{τ̂}_{strat}$. 
</div>

Example

In [35]:
# split the e_hat into L parts
def split(list_to_divide, L):
    k, m = divmod(len(list_to_divide), L)
    return (list_to_divide[i*k + min(i, m):(i+1)*k + min(i+1, m)] for i in range(L))

In [55]:
import math
n = 400
rhos = [0.1, 0.2, 0.3, 0.4]

for rho in rhos:
  L = math.floor(n**rho)
  idx_sorted = sorted(range(len(e_hat)), key=lambda i: e_hat[i])
  idx_stra = list(split(idx_sorted, L))
  l_diff = []
  for index_group in idx_stra:
    y_stra = Y[index_group]
    w_stra = W[index_group]
    y0 = y_stra[w_stra == 0]
    y1 = y_stra[w_stra == 1]
    diff_stra = y1.mean() - y0.mean()
    l_diff.append(diff_stra)

  tau_stra = np.mean(l_diff)
  print(f"When rho = {rho}, Tau_straification: {tau_stra:.3f}")
  print(f"In this case, bias = {tau_stra-0.25:.3f}, MSE = {(tau_stra-0.25)**2:.3f}")

When rho = 0.1, Tau_straification: 0.660
In this case, bias = 0.410, MSE = 0.168
When rho = 0.2, Tau_straification: 0.407
In this case, bias = 0.157, MSE = 0.025
When rho = 0.3, Tau_straification: 0.315
In this case, bias = 0.065, MSE = 0.004
When rho = 0.4, Tau_straification: 0.322
In this case, bias = 0.072, MSE = 0.005


Write your answer here:

$\widehat{τ̂}_{strat}$ = 0.66 when rho = 0.1;

$\widehat{τ̂}_{strat}$ = 0.41 when rho = 0.2;

$\widehat{τ̂}_{strat}$ = 0.31 when rho = 0.3;

$\widehat{τ̂}_{strat}$ = 0.32 when rho = 0.4;

<div class="alert alert-block alert-success">

**Question 5.** Compute the bias and the MSE of IPW and Stratification estimators.
</div>

In [56]:
# For IPW:

bias_IPW = tau_IPW - 0.25
mse_IPW = (tau_IPW - 0.25)**2
print(f"For IPW estimator, bias = {bias_IPW:.3f}, MSE = {mse_IPW:.3f}")

For IPW estimator, bias = 0.052, MSE = 0.003


Write your answer here:

For IPW estimator: Bias = 0.052; MSE = 0.003;

For Stratification estimator, when rho = 0.1: Bias = 0.410; MSE = 0.168;

For Stratification estimator, when rho = 0.2: Bias = 0.157; MSE = 0.025;


For Stratification estimator, when rho = 0.3: Bias = 0.065; MSE = 0.004;


For Stratification estimator, when rho = 0.4: Bias = 0.072; MSE = 0.005;


<div class="alert alert-block alert-success">

**Question 6.**  Repeat the experiment 1000 times and present either the boxplot of the results
or the mean of the results. Comment 
</div>

In [85]:
tau_IPW_list = []
tau_rho_01_list = []
tau_rho_02_list = []
tau_rho_03_list = []
tau_rho_04_list = []

for _ in range(1000):
  X = np.random.uniform(low=-1.0, high=1.0, size=(400,10))
  propensity=0.1+0.85*np.sqrt(np.maximum(0,1+X[:,0]+ X[:,1])/3)
  W =np.random.binomial(1, propensity, 400)
  Y = np.multiply(W, np.maximum(0, X[:,0])) + np.exp(X[:,1] + X[:,2])

  # IPW estimator
  propensity_model = sklearn_linear_model.LogisticRegression(random_state=0, fit_intercept=True).fit(X[:, :2], W)
  estimates_prob = propensity_model.predict_proba(X[:, :2])[:, 1]
  e_hat = clip_probabilities(estimates_prob)
  tau_IPW = (Y * ((W/e_hat) - (1-W)/(1-e_hat))).mean()

  # Stratification estimator
  n = 400
  rhos = [0.1, 0.2, 0.3, 0.4]
  for rho in rhos:
    L = math.floor(n**rho)
    idx_sorted = sorted(range(len(e_hat)), key=lambda i: e_hat[i])
    idx_stra = list(split(idx_sorted, L))
    l_diff = []
    for index_group in idx_stra:
      y_stra = Y[index_group]
      w_stra = W[index_group]
      y0 = y_stra[w_stra == 0]
      y1 = y_stra[w_stra == 1]
      diff_stra = y1.mean() - y0.mean()
      l_diff.append(diff_stra)
    tau_stra = np.mean(l_diff)
    if rho == 0.1 and not np.isnan(tau_stra):
      tau_rho_01_list.append(tau_stra)
    if rho == 0.2 and not np.isnan(tau_stra):
      tau_rho_02_list.append(tau_stra)
    if rho == 0.3 and not np.isnan(tau_stra):
      tau_rho_03_list.append(tau_stra)
    if rho == 0.4 and not np.isnan(tau_stra):
      tau_rho_04_list.append(tau_stra)

  tau_IPW_list.append(tau_IPW)


In [86]:
print(f"The mean tau of IPW estimator is {np.mean(tau_IPW_list):.3f}")
print(f"The mean tau of Stratification (rho=0.1) estimator is {np.mean(tau_rho_01_list):.3f}")
print(f"The mean tau of Stratification (rho=0.2) estimator is {np.mean(tau_rho_02_list):.3f}")
print(f"The mean tau of Stratification (rho=0.3) estimator is {np.mean(tau_rho_03_list):.3f}")
print(f"The mean tau of Stratification (rho=0.4) estimator is {np.mean(tau_rho_04_list):.3f}")

The mean tau of IPW estimator is 0.234
The mean tau of Stratification (rho=0.1) estimator is 0.796
The mean tau of Stratification (rho=0.2) estimator is 0.355
The mean tau of Stratification (rho=0.3) estimator is 0.279
The mean tau of Stratification (rho=0.4) estimator is 0.259


The Stratification estimator achieves a good estimation of ATE on average, better than IPW, when rho is high (when L is large enough). 

But when rho is small, the stratification estimation is worse than IPW estimator.